In [74]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
# import coint
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

In [75]:
currency_dict = {
    'EUR': [],
    'USD': [],
    'JPY': [],
    'GBP': [],
    'CHF': [],
    'CAD': [],
    'AUD': [],
    'NZD': []
}

def get_rates(pair1, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, mt5.TIMEFRAME_M15, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1['close']

In [76]:
symbols = ['EUR', 'GBP', 'USD', 'JPY', 'CHF', 'AUD', 'CAD', 'NZD']
currency_pairs = []
for i in range(len(symbols)):
    for j in range(i+1, len(symbols)):
        pair = symbols[i] + symbols[j]
        currency_pairs.append(pair + '.a')

In [77]:
valid_pairs = []

for pair in currency_pairs:
    base_currency, quote_currency = pair[:3], pair[3:6]

    # Check if the pair is valid
    rates = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_M15, 0, 1)
    if rates is not None:
        valid_pairs.append(pair)
    else:
        # Try switching the currencies and check if the pair is valid
        switched_pair = quote_currency + base_currency + '.a'
        rates = mt5.copy_rates_from_pos(switched_pair, mt5.TIMEFRAME_M15, 0, 1)
        if rates is not None:
            valid_pairs.append(switched_pair)

In [78]:

data = {}
for symbol in valid_pairs:
    data[symbol] = get_rates(symbol, 24)

for symbol, price in data.items():
    # Iterate over the dictionary instead of using multiple if/elif statements
    for currency in currency_dict:
        if currency in symbol:
            currency_dict[currency].append((symbol, price))

average_rates = {}

for base_currency, pairs in currency_dict.items():
    total_sum_for_all_pairs = 0
    for pair_name, pair_data in pairs:
        total_sum_for_all_pairs += pair_data.sum()
    average_rate_for_base_currency = total_sum_for_all_pairs / (len(pairs) * len(pair_data))
    average_rates[base_currency] = round(average_rate_for_base_currency, 2)

print(average_rates)

{'EUR': 22.69, 'USD': 20.85, 'JPY': 129.56, 'GBP': 26.49, 'CHF': 22.67, 'CAD': 15.86, 'AUD': 14.51, 'NZD': 13.3}


In [79]:
for key, value in average_rates.items():
    series = pd.Series(value)
    series.pct_change()
    print(series)

0    22.69
dtype: float64
0    20.85
dtype: float64
0    129.56
dtype: float64
0    26.49
dtype: float64
0    22.67
dtype: float64
0    15.86
dtype: float64
0    14.51
dtype: float64
0    13.3
dtype: float64


In [97]:
# Step 1: Import libraries
import MetaTrader5 as mt5
import pandas as pd

# Step 2: Define necessary data structures and functions
currency_dict = {
    'EUR': [],
    'USD': [],
    'JPY': [],
    'GBP': [],
    'CHF': [],
    'CAD': [],
    'AUD': [],
    'NZD': []
}

def get_rates(pair1, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, mt5.TIMEFRAME_H1, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1['close']

# Step 3: Identify valid pairs
symbols = ['EUR', 'GBP', 'USD', 'JPY', 'CHF', 'AUD', 'CAD', 'NZD']
currency_pairs = []
for i in range(len(symbols)):
    for j in range(i+1, len(symbols)):
        pair = symbols[i] + symbols[j]
        currency_pairs.append(pair + '.a')

valid_pairs = []

for pair in currency_pairs:
    base_currency, quote_currency = pair[:3], pair[3:6]
    # Check if the pair is valid
    rates = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H1, 0, 1)
    if rates is not None:
        valid_pairs.append(pair)
    else:
        # Try switching the currencies and check if the pair is valid
        switched_pair = quote_currency + base_currency + '.a'
        rates = mt5.copy_rates_from_pos(switched_pair, mt5.TIMEFRAME_H1, 0, 1)
        if rates is not None:
            valid_pairs.append(switched_pair)

# Step 4: Store each symbol's price history in dictionary
data = {}
for symbol in valid_pairs:
    data[symbol] = get_rates(symbol, 36)

# Step 5: Calculate and store average rate for each period
average_rates_df_dict = {}

for period in range(36):
    average_rates = {}

    for base_currency in currency_dict.keys():
        total_sum_for_all_pairs = 0
        total_pairs = 0
        for symbol, price_history in data.items():
            if base_currency in symbol:
                total_sum_for_all_pairs += price_history[period]
                total_pairs += 1
        average_rate_for_base_currency = total_sum_for_all_pairs / total_pairs
        average_rates[base_currency] = round(average_rate_for_base_currency, 2)

    average_rates_df_dict[period] = pd.DataFrame.from_records([average_rates])

# Step 6: Concatenate all dataframes into a single dataframe
average_rates_df = pd.concat(average_rates_df_dict, axis=0)
average_rates_df.index = range(len(average_rates_df))

rates = (average_rates_df.pct_change() * 1000)
rates

,EUR,USD,JPY,GBP,CHF,CAD,AUD,NZD
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.443262,-0.481696,0.621118,0.381534,0.000000,0.000000,2.083333,2.279635
2,0.886132,0.963855,0.232775,0.381388,-0.883392,-0.628931,0.693001,0.758150
3,0.000000,-0.481464,-0.077574,0.000000,0.884173,0.000000,-0.692521,-0.757576
4,1.770695,0.963391,1.551590,3.431186,-0.883392,0.000000,2.079002,3.032600
5,-0.441891,-0.481232,0.077459,-0.759878,0.884173,0.629327,0.000000,0.000000
6,0.884173,0.000000,0.154907,0.000000,0.000000,-0.628931,0.691563,0.000000
7,0.000000,0.000000,-0.232324,0.380228,0.000000,-0.629327,0.000000,-0.755858
8,-0.441696,0.000000,0.232378,0.380084,0.441696,0.629723,0.000000,0.756430
9,-0.883783,0.000000,0.077441,0.000000,0.883002,0.000000,0.000000,0.000000


In [106]:
net_changes = {}
net_changes['EUR'] = rates['EUR'].sum()

net_changes

{'EUR': 6.636779060285702}

In [112]:
for col in rates.columns:
    net_changes[col] = rates[col].sum()
net_changes

{'EUR': 6.636779060285702,
 'USD': 0.9832954053461407,
 'JPY': 9.127583322192434,
 'GBP': 12.907023181303877,
 'CHF': 10.566559440567612,
 'CAD': 5.661520595458769,
 'AUD': 10.37970168586988,
 'NZD': 15.107117386641722}

In [115]:
sorted_rates = sorted(net_changes.items(), key=lambda x: x[1])
sorted_rates

[('USD', 0.9832954053461407),
 ('CAD', 5.661520595458769),
 ('EUR', 6.636779060285702),
 ('JPY', 9.127583322192434),
 ('AUD', 10.37970168586988),
 ('CHF', 10.566559440567612),
 ('GBP', 12.907023181303877),
 ('NZD', 15.107117386641722)]

In [164]:
def order_logic():
    pair1 = (sorted_rates[-1][0] + sorted_rates[0][0] + '.a')
    if mt5.symbol_info(pair1) == None:
        pair1 = (sorted_rates[0][0] + sorted_rates[-1][0] + '.a')
        sell_order(pair1)
    else:
        buy_order(pair1)
    
    pair2 = (sorted_rates[-2][0] + sorted_rates[1][0] + '.a')
    if mt5.symbol_info(pair1) == None:
        pair2 = (sorted_rates[1][1] + sorted_rates[-2][-1] + '.a')
        sell_order(pair2)
    else:
        buy_order(pair1)
        

In [162]:
def buy_order(symbol):
    price = mt5.symbol_info_tick(symbol).ask
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "momentum",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1
    
def sell_order(symbol):
    price = mt5.symbol_info_tick(symbol).bid
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": 1.00,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "momentum",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result1 = mt5.order_send(request)
    result1
    
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'pytohn script close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result1 = mt5.order_send(request)